In [15]:
import sys
sys.path.append('./src')
import torch
from src.model_config import ModelConfig, model_collections
from src.bio_utils import seqlogo_from_msa
import data
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import logomaker as lm
from scipy.stats import pearsonr, mode
from math import sqrt, floor, ceil
from tqdm import tqdm
from collections import Counter
import sys
import torch
from src.data import NormalDataset, BERTDataset,LRDataset
from torch.utils.data.dataloader import DataLoader
import torch.nn.functional as F

from pathlib import Path
from tqdm import tqdm
import seaborn as sns
from scipy.stats import pearsonr


# Import models

In [29]:
# Define the model
model_name="deepcnnlstm_in"
mconf = ModelConfig(model_name)
model = model_collections[model_name](mconf)

activation = {}

# Hook function
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

# Load data 
model.load_state_dict(torch.load(Path("./model/in.0410.deepcnnlstm.params.train.13547_5")))
model.eval()
print(model)

DeepCNN_biLSTM(
  (convs): ModuleList(
    (0): Sequential(
      (0): Conv1d(4, 128, kernel_size=(5,), stride=(1,), padding=(2,))
      (1): ReLU()
      (2): Dropout(p=0.1, inplace=False)
    )
    (1): Sequential(
      (0): Conv1d(128, 128, kernel_size=(9,), stride=(1,), padding=(4,))
      (1): ReLU()
      (2): Dropout(p=0.2, inplace=False)
    )
    (2): Sequential(
      (0): Conv1d(128, 128, kernel_size=(11,), stride=(1,), padding=(5,))
      (1): ReLU()
      (2): Dropout(p=0.2, inplace=False)
    )
  )
  (lstm): LSTM(128, 128, num_layers=3, bidirectional=True)
  (fc): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=256, out_features=64, bias=True)
      (1): ReLU()
      (2): Dropout(p=0, inplace=False)
    )
  )
  (reg): Linear(in_features=64, out_features=1, bias=True)
)


In [19]:
# Define the model
model_name="deepcnnlstm_in"
mconf = ModelConfig(model_name)
model = model_collections[model_name](mconf)

activation = {}

# Hook function
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

# Load data 
model.load_state_dict(torch.load(Path("./model/12.0617.deepcnnlstm_in.params.train.5932_5")))
model.eval()
print(model)

DeepCNN_biLSTM(
  (convs): ModuleList(
    (0): Sequential(
      (0): Conv1d(4, 128, kernel_size=(5,), stride=(1,), padding=(2,))
      (1): ReLU()
      (2): Dropout(p=0.1, inplace=False)
    )
    (1): Sequential(
      (0): Conv1d(128, 128, kernel_size=(9,), stride=(1,), padding=(4,))
      (1): ReLU()
      (2): Dropout(p=0.2, inplace=False)
    )
    (2): Sequential(
      (0): Conv1d(128, 128, kernel_size=(11,), stride=(1,), padding=(5,))
      (1): ReLU()
      (2): Dropout(p=0.2, inplace=False)
    )
  )
  (lstm): LSTM(128, 128, num_layers=3, bidirectional=True)
  (fc): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=256, out_features=64, bias=True)
      (1): ReLU()
      (2): Dropout(p=0, inplace=False)
    )
  )
  (reg): Linear(in_features=64, out_features=1, bias=True)
)


# Model Prediction

In [ ]:
virus_name = "in"
model_name = "deepcnnlstm_in"
test_path ="./data/wt_filter_gate_test.csv"
data_size = len(pd.read_csv(test_path))
if model_name == "bert":
    test_data = BERTDataset(test_path, 26,pretrain=False)
elif model_name=="lr":
    test_data = LRDataset(test_path, kmer=3)
else:
    test_data = NormalDataset(test_path)

loader = DataLoader(test_data, batch_size=data_size)


# Define the model
mconf = ModelConfig(model_name)
model = model_collections[model_name](mconf)

model.load_state_dict(torch.load("./model/in.0410.deepcnnlstm.params.train.13547_5"),strict=False)
model.eval()
with torch.no_grad():
    for batch in loader:
        out, loss = model(*batch)

out = out.cpu().numpy()
y_true = batch[1].cpu().numpy()
print("Number of test data: %d"%(len(out)))
r, _ = pearsonr(out, y_true)
print(r)

_, ax = plt.subplots(1, 1, figsize=(4, 4))
ax.scatter(y_true+1, out+1, alpha=0.3, color='gray', s=50)
ax.plot([min(y_true), max(y_true)+1], [min(y_true), max(y_true)+1], linewidth=2, ls='--', color='k')
ax.text(min(y_true), max(y_true), "Pearson $r^2=%.3f$"%r**2, fontsize=12)
ax.text(min(y_true), max(y_true)-1, "Pearson r=%.3f" %r, fontsize=12)
ax.text(min(y_true), max(y_true)-2, "n=%d" %len(out), fontsize=12)
ax.set_xlabel('Measured score')
ax.set_ylabel('Predicted score')
plt.rcParams['svg.fonttype']= 'none'
plt.savefig("./results/test.svg", format='svg', bbox_inches='tight')
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['top'].set_visible(False)

# Motifs

## Extract all motifs

In [ ]:
# Import test set
import pandas as pd
test_path ="./data/wt_filter_gate_test.csv"
data_size = len(pd.read_csv(test_path))
test_data = data.NormalDataset(test_path)
loader = DataLoader(test_data, batch_size=data_size)
filter_sizes = ModelConfig(model_name).conv_filter_sizes
retrieve_layer = "conv1" 
if retrieve_layer == "conv1":
    model.convs[0][0].register_forward_hook(get_activation("conv1"))
    filter_size = filter_sizes[0]
elif retrieve_layer == "conv2":
    model.convs[1][0].register_forward_hook(get_activation("conv2"))
    filter_size = filter_sizes[1] # + filter_sizes[0] // 2
elif retrieve_layer == "conv3":
    model.convs[2][0].register_forward_hook(get_activation("conv3"))
    filter_size = filter_sizes[2] # + filter_sizes[1] // 2
    # filter_size = 13

# Forward the test set
with torch.no_grad():
    for batch in loader:
        out, loss = model(*batch)

first_layer_output = activation[retrieve_layer].data.numpy()

# Find maximal activation
max_activation_val = first_layer_output.max()


# Input data
X = batch[0].cpu().numpy()
y = batch[1].cpu().numpy()
kernel_scores = np.array([
    pearsonr(np.max(first_layer_output[:, i, :], axis=1), y)[0]
     for i in range(first_layer_output.shape[1])
])

# Traverse every kernel
kernel_data_dict = []

df = pd.read_csv("../data/alphavirus/INFL_RandomMutants_Final.csv")
bg_seqs = df["seq"].tolist()

def generate_positional_counts_mat(seqs, centers, filter_size):
    seqs = bg_seqs
    counts_df_group = []
    center_dict = dict(Counter(centers))
    for i in range(filter_size):
        nt_counts_all = {'A': 0, 'U': 0, 'C': 0, 'G': 0}
        for center in center_dict.keys():
            nt_counts = dict(Counter([s[center - filter_size // 2 + i] for s in seqs]))
            for nt in nt_counts_all.keys():
                nt_counts_all[nt] += nt_counts[nt] * center_dict[center]
        counts_df_group.append(pd.DataFrame.from_records([nt_counts_all]))
    counts_df = pd.concat(counts_df_group).reset_index(drop=True)
    return counts_df

for i in tqdm(range(first_layer_output.shape[1])):
    kernel_output = first_layer_output[:, i, :]
    max_val_locations = np.array(
        [np.argmax(kernel_output[j, :]) for j in range(kernel_output.shape[0])]
    )
    max_activation_vals = np.max(kernel_output, axis=1)
    # print(max_activation_vals)

    # Only focus on the sequences which result in activation of 0.7 * maximum
    indices = np.where(max_activation_vals > 0.2 * max_activation_val)[0]

    # Convert embeddings back to sequences
    seqs = [''.join([test_data.corpus[i] for i in X[idx, :]]) for idx in range(X.shape[0])]
    motifs = []
    centers = []
    for j, center in enumerate(max_val_locations[indices]):
        if center >= filter_size // 2 and center < len(seqs[0]) - filter_size // 2:
            motif = seqs[j][center - filter_size // 2 : center + filter_size // 2 + 1]
            if len(motif) == filter_size:  # Ensure the motif length is correct
                motifs.append(motif)
                centers.append(center)


    if len(motifs) < 50:
        counts_df = pd.DataFrame(0, index=range(filter_size), columns=["A", "U", "C", "G"])
        kernel_data_dict.append((counts_df, i, 0, 0, 0)) 
    else:
        bg_counts_df = generate_positional_counts_mat(bg_seqs, centers, filter_size)
        # bg_counts_df = bg_counts_df[['A', 'C', 'G', 'T']]
        
        # print(centers)
        counts_df = seqlogo_from_msa(motifs, bg_counts_mat=bg_counts_df) 
        kernel_data_dict.append((counts_df, i, kernel_scores[i], mode(centers)[0], len(motifs)))
        # kernel_data_dict.append((bg_counts_df, i, kernel_scores[i], mode(centers)[0], len(motifs)))
        
        # print(mode(centers)[0])

# sort motif on location 
def takeSecond(elem):
    return elem[1]
kernel_data_dict.sort(key=takeSecond)
print(kernel_data_dict)
print(len(kernel_data_dict))


In [20]:
flat_data = []

for counts_df, kernel_id, score, mode_center, motif_count in kernel_data_dict:
    for position in range(len(counts_df)):
        row = counts_df.iloc[position]
        flat_data.append({
            "kernel_id": kernel_id,
            "position": position,
            "score": score,
            "mode_center": mode_center,
            "motif_count": motif_count,
            "A": row["A"],
            "U": row["U"],
            "C": row["C"],
            "G": row["G"]
        })


kernel_df = pd.DataFrame(flat_data)
kernel_df.to_csv("./results/kernel_motifs_WT.csv", index=False)


In [ ]:
# Import test set
import pandas as pd
test_path ="./data/12_filter_gate_test.csv"
data_size = len(pd.read_csv(test_path))
test_data = data.NormalDataset(test_path)
loader = DataLoader(test_data, batch_size=data_size)
filter_sizes = ModelConfig(model_name).conv_filter_sizes
retrieve_layer = "conv1" 
if retrieve_layer == "conv1":
    model.convs[0][0].register_forward_hook(get_activation("conv1"))
    filter_size = filter_sizes[0]
elif retrieve_layer == "conv2":
    model.convs[1][0].register_forward_hook(get_activation("conv2"))
    filter_size = filter_sizes[1] # + filter_sizes[0] // 2
elif retrieve_layer == "conv3":
    model.convs[2][0].register_forward_hook(get_activation("conv3"))
    filter_size = filter_sizes[2] # + filter_sizes[1] // 2
    # filter_size = 13

# Forward the test set
with torch.no_grad():
    for batch in loader:
        out, loss = model(*batch)

first_layer_output = activation[retrieve_layer].data.numpy()

# Find maximal activation
max_activation_val = first_layer_output.max()


# Input data
X = batch[0].cpu().numpy()
y = batch[1].cpu().numpy()
kernel_scores = np.array([
    pearsonr(np.max(first_layer_output[:, i, :], axis=1), y)[0]
     for i in range(first_layer_output.shape[1])
])

# Traverse every kernel
kernel_data_dict2 = []
df = pd.read_csv("../data/alphavirus/INFL_RandomMutants_Final.csv")
bg_seqs = df["seq"].tolist()

def generate_positional_counts_mat(seqs, centers, filter_size):
    seqs = bg_seqs
    counts_df_group = []
    center_dict = dict(Counter(centers))
    for i in range(filter_size):
        nt_counts_all = {'A': 0, 'U': 0, 'C': 0, 'G': 0}
        for center in center_dict.keys():
            nt_counts = dict(Counter([s[center - filter_size // 2 + i] for s in seqs]))
            for nt in nt_counts_all.keys():
                nt_counts_all[nt] += nt_counts[nt] * center_dict[center]
        counts_df_group.append(pd.DataFrame.from_records([nt_counts_all]))
    counts_df = pd.concat(counts_df_group).reset_index(drop=True)
    return counts_df

for i in tqdm(range(first_layer_output.shape[1])):
    kernel_output = first_layer_output[:, i, :]
    max_val_locations = np.array(
        [np.argmax(kernel_output[j, :]) for j in range(kernel_output.shape[0])]
    )
    max_activation_vals = np.max(kernel_output, axis=1)
    # print(max_activation_vals)

    # Only focus on the sequences which result in activation of 0.7 * maximum
    indices = np.where(max_activation_vals > 0.2 * max_activation_val)[0]

    # Convert embeddings back to sequences
    seqs = [''.join([test_data.corpus[i] for i in X[idx, :]]) for idx in range(X.shape[0])]
    motifs = []
    centers = []
    for j, center in enumerate(max_val_locations[indices]):
        if center >= filter_size // 2 and center < len(seqs[0]) - filter_size // 2:
            motif = seqs[j][center - filter_size // 2 : center + filter_size // 2 + 1]
            if len(motif) == filter_size:  # Ensure the motif length is correct
                motifs.append(motif)
                centers.append(center)


    if len(motifs) < 50:
        counts_df = pd.DataFrame(0, index=range(filter_size), columns=["A", "U", "C", "G"])
        kernel_data_dict2.append((counts_df, i, 0, 0, 0)) 
    else:
        bg_counts_df = generate_positional_counts_mat(bg_seqs, centers, filter_size)
        # bg_counts_df = bg_counts_df[['A', 'C', 'G', 'T']]
        
        # print(centers)
        counts_df = seqlogo_from_msa(motifs, bg_counts_mat=bg_counts_df) 
        kernel_data_dict2.append((counts_df, i, kernel_scores[i], mode(centers)[0], len(motifs)))
        # kernel_data_dict.append((bg_counts_df, i, kernel_scores[i], mode(centers)[0], len(motifs)))
        
        # print(mode(centers)[0])

# sort motif on location 
def takeSecond(elem):
    return elem[1]
kernel_data_dict2.sort(key=takeSecond)
print(kernel_data_dict2)
print(len(kernel_data_dict2))


In [22]:
flat_data = []

for counts_df, kernel_id, score, mode_center, motif_count in kernel_data_dict2:
    for position in range(len(counts_df)):
        row = counts_df.iloc[position]
        flat_data.append({
            "kernel_id": kernel_id,
            "position": position,
            "score": score,
            "mode_center": mode_center,
            "motif_count": motif_count,
            "A": row["A"],
            "U": row["U"],
            "C": row["C"],
            "G": row["G"]
        })


kernel_df = pd.DataFrame(flat_data)
kernel_df.to_csv("./results/kernel_motifs_12.csv", index=False)


# PHATE

In [ ]:
import numpy as np
import phate
import matplotlib.pyplot as plt
from matplotlib import cm  

kernel_data_dict= [item for item in kernel_data_dict if item[3] != 0]
kernel_data_dict2= [item for item in kernel_data_dict2 if item[3] != 0]

features1 = [np.append(item[0].values.flatten(), item[3]*0.4) for item in kernel_data_dict]
features2 = [np.append(item[0].values.flatten(), item[3]*0.4) for item in kernel_data_dict2]
features = np.concatenate([features1, features2], axis=0)


phate_operator = phate.PHATE(knn=3, decay=10, t=5)
phate_embedding = phate_operator.fit_transform(features)

scores1 = [item[2] for item in kernel_data_dict]
scores2 = [item[2] for item in kernel_data_dict2]
scores = scores1 + scores2  
plt.figure(figsize=(12, 10))
norm1 = plt.Normalize(vmin=min(scores1), vmax=max(scores1))  
norm2 = plt.Normalize(vmin=min(scores2), vmax=max(scores2))  

cmap1 = cm.Blues 
cmap2 = cm.Reds  

scatter1 = plt.scatter(phate_embedding[:len(scores1), 0], phate_embedding[:len(scores1), 1], 
                       c=scores1, cmap=cmap1, norm=norm1, s=50)

scatter2 = plt.scatter(phate_embedding[len(scores1):, 0], phate_embedding[len(scores1):, 1], 
                       c=scores2, cmap=cmap2, norm=norm2,  s=50)

plt.xlabel("PHATE 1")
plt.ylabel("PHATE 2")
'''
for i in range(len(kernel_data_dict)):
    plt.text(phate_embedding[i, 0], phate_embedding[i, 1], 
             f" {kernel_data_dict[i][1]}", fontsize=10, color='black')
'''

"""for i in range(len(kernel_data_dict)):
    plt.text(phate_embedding[i, 0], phate_embedding[i, 1], 
             f" {kernel_data_dict[i][1]}", fontsize=10, color='black')
"""

"""for i in range(len(kernel_data_dict)):
    if norm1(scores1[i]) >0.5:
        plt.text(
            phate_embedding[i, 0], 
            phate_embedding[i, 1], 
            f" {kernel_data_dict[i][1]}", 
            fontsize=10, color='blue'
        )

for i in range(len(kernel_data_dict2)):
    if norm2(scores2[i]) >0.5:
        plt.text(
            phate_embedding[len(scores1) + i, 0], 
            phate_embedding[len(scores1) + i, 1], 
            f" {kernel_data_dict2[i][1]}", 
            fontsize=10, color='red'
        )"""
legend1_color = cmap1(norm1(max(scores1) - 0.2)) 
legend2_color = cmap2(norm2(max(scores2) - 0.2))  


plt.scatter([], [], color=legend1_color, label='WT', s=50)  
plt.scatter([], [], color=legend2_color, label='mut 1', s=50)  
plt.legend(fontsize=15)
plt.rcParams['svg.fonttype'] = 'none'
plt.savefig('./results/phate_12.svg', format='svg', bbox_inches='tight')
plt.show()


In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

n_clusters = 20
kmeans = KMeans(
    n_clusters=n_clusters,
    init='k-means++',    
    n_init=50,           # 随机初始化，防止局部最优
    max_iter=2000,        
    tol=1e-9,            # 收敛容忍度
    random_state=42
)

cluster_labels = kmeans.fit_predict(phate_embedding)
score = silhouette_score(phate_embedding, cluster_labels)
print(f"KMeans Silhouette Score: {score:.4f}")

cmap = cm.get_cmap('tab20', n_clusters)

# plot
plt.figure(figsize=(12, 10))
scatter = plt.scatter(phate_embedding[:, 0], phate_embedding[:, 1],
                      c=cluster_labels, cmap=cmap, s=50, alpha=0.8)
plt.xlabel("PHATE 1")
plt.ylabel("PHATE 2")

legend_elements = [
    Patch(facecolor=cmap(i), label=f"Cluster {i}") for i in range(n_clusters)
]
plt.legend(handles=legend_elements, title="Cluster ID", fontsize=12,
           bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid(True, linestyle='--', alpha=0.3)
plt.tight_layout()
plt.rcParams['svg.fonttype'] = 'none'
plt.savefig('./results/phate_12_cluster.svg', format='svg', bbox_inches='tight')
plt.show()


# Importance

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
kernel_data_dict= [item for item in kernel_data_dict if item[3] != 0]
centers = [item[3] + 1 for item in kernel_data_dict]
scores = []
labels = []
for item in kernel_data_dict:
    labels.append(item[1])
    scores.append(item[2])
norm = plt.Normalize(vmin=min(scores), vmax=max(scores))
plt.figure(figsize=(12, 8))
scatter = plt.scatter(
    centers, scores, 
    c=scores, cmap=cm.Blues, norm=norm, s=50, label='Transcription'
)
"""for x, y, label in zip(centers2, scores2, labels):
    plt.text(x, y, f"{label}", fontsize=10, ha='center', va='bottom')"""

all_centers = sorted(set(centers)) 
plt.xticks(all_centers, fontsize=8) 
cbar = plt.colorbar(scatter, cmap=cm.coolwarm, norm=norm)
cbar.set_label('filter importance')

plt.xlabel("Center", fontsize=14)
plt.ylabel("importance", fontsize=14)
plt.rcParams['svg.fonttype']= 'none'
plt.savefig('./results/WT_importance.svg', format='svg', bbox_inches='tight')

plt.show()


## Draw all motifs

In [34]:
def takeSecond(elem):
    return elem[1]
kernel_data_dict.sort(key=takeSecond)

In [ ]:
n_kernel = len(kernel_data_dict)
row = floor(sqrt(n_kernel))
col = ceil(n_kernel / row)

fig = plt.figure(figsize=(col * 4, row * 4))
gs = plt.GridSpec(row * 2, col, figure=fig, height_ratios=[10, 1] * row)

for i, (counts_df, kernel_idx, kernel_score, kernel_loc, n_motifs) in enumerate(kernel_data_dict[:n_kernel]):
    if counts_df.empty or counts_df.shape[0] == 0:
        print(f"Skipping kernel {kernel_idx} due to empty counts_df.")
        continue
    
    ax = fig.add_subplot(gs[i // col * 2, i % col])
    lm.Logo(counts_df, ax=ax, color_scheme="classic")
    #ax.set_ylim([0, 0.05])
    max_height = counts_df.sum(axis=1).max()  
    text_y = max_height + 0.09 * max_height
    #text_y = max_height
    #ax.set_xticks(list(range(filter_size)))
    '''ax.text(
        filter_size // 2, 0.97, 
        f"Kernel {kernel_idx}: $r^2\;=\;{kernel_score:.3f}$\ncenter={kernel_loc}, n={n_motifs}",
        ha='center', va='top', fontsize=15
    )'''
    ax.text(
    filter_size // 2+0.02, text_y, 
    f"Kernel {kernel_idx}: center={kernel_loc+1}",ha='center', va='top', fontsize=17
    )
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_yticks([])
    ax.set_xticks([])  
    ax.set_xticklabels([]) 
    #ax2 = fig.add_subplot(gs[i // col * 2 + 1, i % col])
    '''kernel_score_dist = np.array([
        pearsonr(first_layer_output[:, i, loc], y)[0]
        for loc in range(first_layer_output.shape[2])
    ])'''
    #sns.heatmap(kernel_score_dist.reshape((1, -1)), ax=ax2, cmap='seismic', vmin=-1, vmax=1, cbar=False)
    #ax2.axis('off')
plt.rcParams['svg.fonttype']= 'none'
plt.savefig('./results/WT_kernels.svg', format='svg', bbox_inches='tight')


# Saliency map

In [ ]:
import torch
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


def predict_seq_score(seq):
    X = torch.tensor([test_data.tok2idx[s] for s in seq]).unsqueeze(0)
    original_score, _ = model(X,None)
    original_score = original_score.detach().numpy()
    return original_score[0]

# Define the original sequence and compute its score
original_seq = "UCGUUUUCGUCCCACUGUUUUUGUAU"
original_score = predict_seq_score(original_seq)
saliency_arr = np.zeros((4, len(original_seq)))
original="UCGUUUUCGUCCCACUGUUUUUGUAU"

# Compute saliency scores for each nucleotide position
for i in range(len(original_seq)):
    for j, nt in enumerate(['A', 'U', 'C', 'G']):
        saliency_arr[j, i] = predict_seq_score(original_seq[:i] + nt + original_seq[i+1:])
saliency_arr -= original_score

# Define the range for color mapping
vmin = saliency_arr.min()  # Minimum value for the color scale
vmax = saliency_arr.max()  # Maximum value for the color scale

# Plotting
_, ax = plt.subplots(1, 1, figsize=(10, 2))
sns.heatmap(saliency_arr, cmap='coolwarm', center=0, ax=ax, vmin=vmin, vmax=vmax)

# Set the ticks and labels correctly
num_positions = len(original_seq)
ax.set_yticks(np.arange(4) + 0.5)
ax.set_yticklabels(['A', 'U', 'C', 'G'])
ax.set_xticks(np.arange(num_positions) + 0.5)
ax.set_xticklabels(np.arange(1, num_positions + 1), fontsize=10)

# Add black points for the original sequence
original_seq_idx = np.array(['AUCG'.find(nt) for nt in original])
ax.scatter(np.arange(len(original_seq)) + 0.5, original_seq_idx + 0.5, c="k", zorder=5)
plt.rcParams['svg.fonttype']= 'none'
plt.savefig('./results//saliency.svg', format='svg', bbox_inches='tight')
plt.show()


In [37]:

saliency_df = pd.DataFrame(
    saliency_arr,
    index=['A', 'U', 'C', 'G'],
    columns=[f"pos_{i+1}" for i in range(len(original_seq))]
)
saliency_df.to_csv('./results//saliency.csv')
